In [1]:
from modules.loader.DataLoader import DataLoader
from modules.processor.DataProcessor import DataProcessor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw
from lifelines.utils import concordance_index
from sklearn.inspection import permutation_importance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

In [2]:
data_loader = DataLoader()
df_raw = data_loader.load_from_csv()
data_processor = DataProcessor()
df = data_processor.get_df(df_raw)

Subset length: 536
Subset shape: (536, 54)


In [3]:
s_num_fields = ['age', 'BMI', 'DIA_MAX_TUMEUR_1', 'DIA_MAX_TUMEUR_2', 'DIA_MAX_TUMEUR_3'] #, 'DIA_MAX_TUMEUR_4'
s_cat_fields = ['gender', 'FUMEUROUINON', 'ICD9', 'dead'] # , 'PACEMAKER_ON', 'DIABETIQUE_ON', 'INSULO_ON', 'ALLERGIE_ON', 'INSUFFURENALE_ON'
p_num_fields = ['TotalDose', 'NbVolumes'] #, 'TotalSessions'
p_cat_fields = ['DEGRURG', 'CHIMOANT_ON', 'NRSF11', 'OMS', 'MOBPAT'] # 'IRRA_ANT_ON', 'IRRANT_OLOCA', 'CHIRURGIE_ANT_ON', 
c_cat_fields = ['T'] # 'N', 'M'
    

In [4]:
num_features = s_num_fields + p_num_fields
cat_features = s_cat_fields + p_cat_fields + c_cat_fields
cat_features.remove('dead')

In [5]:
cat_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
preprocessor = ColumnTransformer(
    [
        ("numerical", StandardScaler(), num_features),
        ("categorical", cat_preprocessor, cat_features)
        # ("icd9", oh_encoder, ['ICD9']),
    ],
    verbose_feature_names_out=False
)
preprocessor.set_output(transform = "pandas")

,transformers,"[('numerical', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [6]:
features = num_features + cat_features
X = df[features]

Y_time  = df['time_in_months']
Y_event = df['dead']

In [7]:
X = preprocessor.fit_transform(X)

all_indices = df.index.to_numpy()
train_indices, test_indices = train_test_split(
    all_indices, 
    test_size=0.2, 
    random_state=42
)
X_train = X.loc[train_indices]
X_test = X.loc[test_indices]
time_train = Y_time.loc[train_indices]
time_test = Y_time.loc[test_indices]
event_train = Y_event.loc[train_indices]
event_test = Y_event.loc[test_indices]

y_train = Surv.from_arrays(event_train, time_train)
y_test = Surv.from_arrays(event_test, time_test)

In [8]:
dtrain = xgb.DMatrix(X_train, label=time_train)
dtrain.set_float_info('label_lower_bound', time_train)
dtrain.set_float_info('label_upper_bound', np.where(event_train==1, time_train, np.inf))
dtest = xgb.DMatrix(X_test, label=time_test)
dtest.set_float_info('label_lower_bound', time_test)
dtest.set_float_info('label_upper_bound', np.where(event_test==1, time_test, np.inf))

In [9]:
# --- 3. ФУНКЦІЯ СТВОРЕННЯ DMatrix З МЕЖАМИ ЦЕНЗУРУВАННЯ ---
def create_dmatrix(X, time, event):
    dmat = xgb.DMatrix(X, label=time)
    dmat.set_float_info('label_lower_bound', time)
    dmat.set_float_info('label_upper_bound', np.where(event==1, time, np.inf))
    return dmat

In [10]:
monotone = [0] * len(features)
for i, col in enumerate(features):
    if col in ['NbVolumes', 'DIA_MAX_TUMEUR_2']: #'age', 'DIA_MAX_TUMEUR_1', , 'T'
        monotone[i] = 1

In [11]:
params = {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'hist', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 'verbosity': 0, 'reg_alpha': np.float64(0.8073366978931656), 'reg_lambda': np.float64(3.3972828803985493), 'min_child_weight': np.int64(5), 'max_depth': np.int64(3), 'learning_rate': 0.01, 'subsample': np.float64(0.9106576696725992), 'colsample_bytree': np.float64(0.7766276769354553), 
          'aft_loss_distribution': 'logistic'}
#   Фолд 1/5: C-індекс = 0.7178
#   Фолд 2/5: C-індекс = 0.6710
#   Фолд 3/5: C-індекс = 0.7030
#   Фолд 4/5: C-індекс = 0.6789
#   Фолд 5/5: C-індекс = 0.6348
# Середній C-індекс по 5 фолдам: 0.6811
# На повному наборі

In [12]:
params = {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'auto', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 
          'monotone_constraints': (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'verbosity': 0, 'reg_alpha': np.float64(2.6164135794446444), 'reg_lambda': np.float64(2.706328102158079), 'min_child_weight': np.int64(13), 'max_depth': np.int64(5), 'learning_rate': 0.001, 'subsample': np.float64(0.8828030562164276), 'colsample_bytree': np.float64(0.7108104445526267), 
          'aft_loss_distribution': 'logistic'}
# Середній C-індекс: 0.6846 на тренінговій кросвалідації
#   Фолд 1/5: C-індекс = 0.7121
#   Фолд 2/5: C-індекс = 0.6675
#   Фолд 3/5: C-індекс = 0.6964
#   Фолд 4/5: C-індекс = 0.6822
#   Фолд 5/5: C-індекс = 0.6567
# Середній C-індекс по 5 фолдам: 0.6830 на повному наборі

In [13]:
params = {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'subsample': 0.85, 'colsample_bytree': 0.8, 'tree_method': 'hist', 'aft_loss_distribution_scale': 0.79, 'seed': 42, 'verbosity': 0, 'reg_alpha': 0.5, 'reg_lambda': 4.0, 'max_depth': 3, 'min_child_weight': 10, 'learning_rate': 0.005, 
          'aft_loss_distribution': 'logistic'}
#   Фолд 1/5: C-індекс = 0.7107
#   Фолд 2/5: C-індекс = 0.6623
#   Фолд 3/5: C-індекс = 0.7025
#   Фолд 4/5: C-індекс = 0.6686
#   Фолд 5/5: C-індекс = 0.6475
# Середній C-індекс по 5 фолдам: 0.6783
# На повному наборі

In [14]:
params = {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'subsample': 0.85, 'colsample_bytree': 0.8, 'tree_method': 'hist', 'aft_loss_distribution_scale': 0.79, 
          'monotone_constraints': (0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'seed': 42, 'verbosity': 0, 'reg_alpha': 0.1, 'reg_lambda': 4.0, 'max_depth': 5, 'min_child_weight': 10, 'learning_rate': 0.005, 
          'aft_loss_distribution': 'logistic'}
# Середній C-індекс: 0.6850 на тренінговій кросвалідації
#   Фолд 1/5: C-індекс = 0.7045
#   Фолд 2/5: C-індекс = 0.6574
#   Фолд 3/5: C-індекс = 0.7033
#   Фолд 4/5: C-індекс = 0.6667
#   Фолд 5/5: C-індекс = 0.6492
# Середній C-індекс по 5 фолдам: 0.6762 на повному наборі

In [15]:
params = {'objective': 'survival:aft',
        'eval_metric': 'aft-nloglik',
        'aft_loss_distribution': 'extreme',
        'aft_loss_distribution_scale': 0.79,
        'subsample': 0.85,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.5,
        'reg_lambda': 2.0,
        'tree_method': 'hist',
        'learning_rate': 0.005,
        'max_depth': 4,
        'min_child_weight': 20, 
        'seed': 42,
        # 'monotone_constraints': tuple(monotone),
        'verbosity': 0}
#   Фолд 1/5: C-індекс = 0.7274
#   Фолд 2/5: C-індекс = 0.6502
#   Фолд 3/5: C-індекс = 0.6911
#   Фолд 4/5: C-індекс = 0.6639
#   Фолд 5/5: C-індекс = 0.6305
# Середній C-індекс по 5 фолдам: 0.6726
# На повному наборі

In [16]:
bst = xgb.train(params, dtrain, num_boost_round=20000,
            evals=[(dtrain, 'train'), (dtest, 'test')],
            early_stopping_rounds=500,
            verbose_eval=500)

[0]	train-aft-nloglik:24.29163	test-aft-nloglik:23.86947
[500]	train-aft-nloglik:7.68251	test-aft-nloglik:7.62379
[1000]	train-aft-nloglik:2.63203	test-aft-nloglik:2.60629
[1500]	train-aft-nloglik:2.32968	test-aft-nloglik:2.38992
[2000]	train-aft-nloglik:2.24494	test-aft-nloglik:2.37725
[2500]	train-aft-nloglik:2.18908	test-aft-nloglik:2.37745
[2713]	train-aft-nloglik:2.16876	test-aft-nloglik:2.38241


In [17]:
months = bst.predict(dtest)

In [18]:
print(f"XGBoost c-index: {concordance_index(time_test, months, event_test)}")
print(f"IPCW C-index: {concordance_index_ipcw(y_train, y_test, -months)[0]:.4f}")

XGBoost c-index: 0.7191930207197382
IPCW C-index: 0.6523


In [19]:
# Крос-валідація KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    
    # Розділення даних
    # Припускаємо, що X_train, time_train, event_train є об'єктами з індексацією
    X_fold_train, X_fold_test = X.iloc[train_index], X.iloc[test_index]
    time_fold_train, time_fold_test = Y_time.iloc[train_index], Y_time.iloc[test_index]
    event_fold_train, event_fold_test = Y_event.iloc[train_index], Y_event.iloc[test_index]

    # Створення DMatrix
    dtrain = create_dmatrix(X_fold_train, time_fold_train, event_fold_train)
    dtest = create_dmatrix(X_fold_test, time_fold_test, event_fold_test)
    
    # Навчання моделі
    bst = xgb.train(
        params, 
        dtrain, 
        num_boost_round=20000, 
        evals=[(dtest, 'test')],
        early_stopping_rounds=500, # Короткий early_stopping для швидкого пошуку
        verbose_eval=False
    )
    # Прогноз часу
    months_pred = bst.predict(dtest)
    
    # Обчислення C-індексу (lifelines)
    c_index = concordance_index(time_fold_test, months_pred, event_fold_test)
    print(f"  Фолд {fold+1}/5: C-індекс = {c_index:.4f}")
    cv_scores.append(c_index)
    

mean_cv_score = np.mean(cv_scores)
print(f"Середній C-індекс по 5 фолдам: {mean_cv_score:.4f}")

  Фолд 1/5: C-індекс = 0.7249
  Фолд 2/5: C-індекс = 0.6516
  Фолд 3/5: C-індекс = 0.6913
  Фолд 4/5: C-індекс = 0.6618
  Фолд 5/5: C-індекс = 0.6322
Середній C-індекс по 5 фолдам: 0.6724


In [20]:
from sklearn.model_selection import KFold
from scipy.stats import uniform, randint
import random

# --- 1. ПРИПУЩЕННЯ: Ви вже маєте X_train, time_train, event_train (як ndarray або Series/DataFrame) ---

# --- 2. ПРОСТІР ПАРАМЕТРІВ ДЛЯ ПОШУКУ ---
# Розширюємо діапазони навколо ваших поточних параметрів для L1/L2 та глибини
param_dist = {
    # Регуляризація
    'reg_alpha': uniform(0.0, 3.0),      # L1 (Lasso)
    'reg_lambda': uniform(0.5, 5.0),     # L2 (Ridge)
    'min_child_weight': randint(5, 30),  # Дозволяємо менші значення
    
    # Параметри дерева
    'max_depth': randint(3, 6),          # Глибина [3, 4, 5]
    
    # Параметри навчання
    'learning_rate': [0.001, 0.005, 0.01, 0.02], # Крок навчання
    'subsample': uniform(0.7, 0.3),      # Частка зразків [0.7 - 1.0]
    'colsample_bytree': uniform(0.7, 0.3),# Частка ознак [0.7 - 1.0]
    
    # AFT Розподіл (спробуємо різні)
    'aft_loss_distribution': ['extreme', 'normal', 'logistic']
}

# Фіксовані параметри
base_params = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'tree_method': 'hist',
    'seed': 42,
    'aft_loss_distribution_scale': 0.79, # Залишаємо ваше фіксоване значення
    'monotone_constraints': tuple(monotone),
    'verbosity': 0
}

# --- 3. ФУНКЦІЯ СТВОРЕННЯ DMatrix З МЕЖАМИ ЦЕНЗУРУВАННЯ ---
def create_dmatrix(X, time, event):
    dmat = xgb.DMatrix(X, label=time)
    dmat.set_float_info('label_lower_bound', time)
    # Якщо event=1 (подія настала), верхня межа = час. Якщо event=0 (цензуровано), верхня межа = нескінченність.
    dmat.set_float_info('label_upper_bound', np.where(event==1, time, np.inf))
    return dmat

In [21]:
# Налаштування пошуку
N_ITER = 100  # Рекомендовано: 50–100 ітерацій для реальних даних
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

best_score = 0.0
best_params = None

print(f"Початок Randomized Search на {N_ITER} ітерацій...")

# 1. Створення випадкових комбінацій параметрів
random_combinations = []
for _ in range(N_ITER):
    params = base_params.copy()
    for k, v in param_dist.items():
        if isinstance(v, list):
            params[k] = random.choice(v)
        else:
            params[k] = v.rvs(1)[0] # Генерація випадкового числа
    random_combinations.append(params)


# 2. Перебір випадкових комбінацій
for i, current_params in enumerate(random_combinations):
    
    cv_scores = []
    
    # 3. Крос-валідація KFold
    for fold, (train_index, test_index) in enumerate(kf.split(X_train)):
        
        # Розділення даних на фолди (припускаємо, що X_train, time_train, event_train - це Series/DataFrame)
        X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
        time_fold_test = time_train.iloc[test_index]
        event_fold_test = event_train.iloc[test_index]
        
        # Створення DMatrix для навчання
        dtrain = create_dmatrix(X_fold_train, time_train.iloc[train_index], event_train.iloc[train_index])
        dtest = create_dmatrix(X_fold_test, time_fold_test, event_fold_test)
        
        # Навчання моделі
        bst = xgb.train(
            current_params, 
            dtrain, 
            num_boost_round=20000, # Використовуємо велике число, оскільки є early_stopping
            evals=[(dtest, 'test')],
            early_stopping_rounds=500,
            verbose_eval=False
        )
        
        # Прогноз часу
        months_pred = bst.predict(dtest)
        
        # Обчислення C-індексу (lifelines)
        c_index = concordance_index(time_fold_test, months_pred, event_fold_test)
        cv_scores.append(c_index)
        
    
    # Середній показник C-індексу для поточної комбінації
    mean_cv_score = np.mean(cv_scores)
    
    print(f"Ітерація {i+1}/{N_ITER}: C-індекс = {mean_cv_score:.4f}, Параметри: {current_params}")
    
    # Оновлення найкращого результату
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_params = current_params.copy()
        
print("\n--- ПОШУК ЗАВЕРШЕНО ---")
print(f"Найкращий C-індекс крос-валідації: {best_score:.4f}")
print(f"Найкращі параметри: {best_params}")

Початок Randomized Search на 100 ітерацій...
Ітерація 1/100: C-індекс = 0.6628, Параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'hist', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 'monotone_constraints': (0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'verbosity': 0, 'reg_alpha': np.float64(2.674003279244375), 'reg_lambda': np.float64(1.522677601627318), 'min_child_weight': np.int64(24), 'max_depth': np.int64(5), 'learning_rate': 0.02, 'subsample': np.float64(0.8549596387203277), 'colsample_bytree': np.float64(0.920263467478742), 'aft_loss_distribution': 'extreme'}
Ітерація 2/100: C-індекс = 0.6639, Параметри: {'objective': 'survival:aft', 'eval_metric': 'aft-nloglik', 'tree_method': 'hist', 'seed': 42, 'aft_loss_distribution_scale': 0.79, 'monotone_constraints': (0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'verbosity': 0, 'reg_alpha': np.float64(0.50543698105411), 'reg_lambda': np.float64(2.7219101340149456), 'min_child_weight': np.int

In [22]:
import itertools
import time

# --- 1. ПРИПУЩЕННЯ: X_train, time_train, event_train вже визначені як Series/DataFrame/ndarray ---

# --- 2. ГРИД ПАРАМЕТРІВ ДЛЯ ПОШУКУ ---
param_grid = {
    'reg_alpha': [0.1, 0.5, 1.0],         # L1-регуляризація (Ваш 0.5 в середині)
    'reg_lambda': [1.0, 2.0, 4.0],        # L2-регуляризація (Ваш 2.0 в середині)
    'max_depth': [3, 4, 5],               # Глибина (Ваш 4 в середині)
    'min_child_weight': [10, 20, 30],     # Мінімальна вага листка (Ваш 20 в середині)
    'learning_rate': [0.005, 0.01],       # Крок навчання (Ваш 0.005)
    'aft_loss_distribution': ['extreme', 'logistic', 'normal'] # Спробуємо інший розподіл
}

# Фіксовані параметри
base_params = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'subsample': 0.85,
    'colsample_bytree': 0.8,
    'tree_method': 'hist',
    'aft_loss_distribution_scale': 0.79,
    'seed': 42,
    'verbosity': 0
}

In [23]:
# --- 4. НАЛАШТУВАННЯ KFold та ПОШУКУ ---
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
best_score = 0.0
best_params = None
best_model = None
total_combinations = len(list(itertools.product(*param_grid.values())))

print(f"Початок Grid Search на {total_combinations} ітерацій...")
start_time = time.time()

# Генерація всіх комбінацій параметрів
keys, values = zip(*param_grid.items())
grid_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# 5. Перебір усіх комбінацій
for i, current_tuning_params in enumerate(grid_combinations):
    
    current_params = base_params.copy()
    current_params.update(current_tuning_params) # Об'єднання базових і поточних параметрів
    
    cv_scores = []
    model = None
    # Крос-валідація KFold
    for fold, (train_index, test_index) in enumerate(kf.split(X_train)):
        
        # Розділення даних
        # Припускаємо, що X_train, time_train, event_train є об'єктами з індексацією
        X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
        time_fold_train, time_fold_test = time_train.iloc[train_index], time_train.iloc[test_index]
        event_fold_train, event_fold_test = event_train.iloc[train_index], event_train.iloc[test_index]
        
        # Створення DMatrix
        dtrain = create_dmatrix(X_fold_train, time_fold_train, event_fold_train)
        dtest = create_dmatrix(X_fold_test, time_fold_test, event_fold_test)
        
        # Навчання моделі
        bst = xgb.train(
            current_params, 
            dtrain, 
            num_boost_round=10000, 
            evals=[(dtest, 'test')],
            early_stopping_rounds=100, # Короткий early_stopping для швидкого пошуку
            verbose_eval=False
        )
        model = bst
        # Прогноз часу
        months_pred = bst.predict(dtest)
        
        # Обчислення C-індексу (lifelines)
        c_index = concordance_index(time_fold_test, months_pred, event_fold_test)
        cv_scores.append(c_index)
        
    
    mean_cv_score = np.mean(cv_scores)
    
    # Вивід прогресу (опціонально)
    if (i + 1) % 20 == 0 or i == total_combinations - 1:
        print(f"Прогрес {i+1}/{total_combinations}: C-індекс = {mean_cv_score:.4f}")
    
    # Оновлення найкращого результату
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_params = current_params.copy()
        best_model = model
        
end_time = time.time()
print("\n--- ПОШУК ЗАВЕРШЕНО ---")
print(f"Загальний час виконання: {(end_time - start_time):.2f} секунд")
print(f"Найкращий C-індекс крос-валідації (lifelines): {best_score:.4f}")
print(f"Найкращі параметри: {best_params}")

Початок Grid Search на 486 ітерацій...
Прогрес 20/486: C-індекс = 0.6779
Прогрес 40/486: C-індекс = 0.6621
Прогрес 60/486: C-індекс = 0.6650
Прогрес 80/486: C-індекс = 0.6768
Прогрес 100/486: C-індекс = 0.6671
Прогрес 120/486: C-індекс = 0.6634
Прогрес 140/486: C-індекс = 0.5000
Прогрес 160/486: C-індекс = 0.6682
Прогрес 180/486: C-індекс = 0.6627
Прогрес 200/486: C-індекс = 0.6776
Прогрес 220/486: C-індекс = 0.6643
Прогрес 240/486: C-індекс = 0.6630
Прогрес 260/486: C-індекс = 0.6780
Прогрес 280/486: C-індекс = 0.6695
Прогрес 300/486: C-індекс = 0.6620
Прогрес 320/486: C-індекс = 0.5000
Прогрес 340/486: C-індекс = 0.6687
Прогрес 360/486: C-індекс = 0.6571
Прогрес 380/486: C-індекс = 0.6828
Прогрес 400/486: C-індекс = 0.6659
Прогрес 420/486: C-індекс = 0.6620
Прогрес 440/486: C-індекс = 0.6793
Прогрес 460/486: C-індекс = 0.6678
Прогрес 480/486: C-індекс = 0.6647
Прогрес 486/486: C-індекс = 0.6573

--- ПОШУК ЗАВЕРШЕНО ---
Загальний час виконання: 2450.15 секунд
Найкращий C-індекс крос-в